#Bắt đầu làm:

In [ ]:
## file wecode
!gdown --id 1NkcKJyW8XpvaKDZlG-HRCY4igmL1ZSGF
## file score
!gdown --id 1JFZZRDMWNWWWxzkTWzoJI1v8XC7GTZfO

Downloading...
From: https://drive.google.com/uc?id=1NkcKJyW8XpvaKDZlG-HRCY4igmL1ZSGF
To: /content/wecode.csv
30.8MB [00:00, 73.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JFZZRDMWNWWWxzkTWzoJI1v8XC7GTZfO
To: /content/score_training.csv
100% 23.9k/23.9k [00:00<00:00, 20.5MB/s]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("/content/wecode.csv")
score = pd.read_csv('/content/score_training.csv')
score["CK"] = pd.to_numeric(score["CK"],errors = 'coerce') # convert object -> float64
score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      975 non-null    int64  
 1   QT      973 non-null    object 
 2   GK      974 non-null    object 
 3   TH      974 non-null    object 
 4   CK      920 non-null    float64
 5   TBM     974 non-null    object 
 6   TBTL    974 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 53.4+ KB


In [ ]:
# lấy cột user_id
train = pd.DataFrame(np.sort(data.user_id.unique()), columns= ['user_id'])

In [ ]:
# Tổng pre_score của các submssion có is_final = 1

d = data[data.is_final == 1].groupby(['user_id'], as_index= False)[['pre_score']].sum()

train = train.merge(d,on="user_id")



In [ ]:
wecode_data_new = data

# Hàm kiểm tra code đã solved chưa
def checkSolved(data):
  if (data['is_final'] == 1) and (data['pre_score'] == 10000):
    return 1
  return 0
wecode_data_new['solved'] = wecode_data_new.apply(checkSolved, axis=1)
groupByUserID = wecode_data_new.groupby(by='user_id', sort = True)
problem_solved= groupByUserID['solved'].apply(lambda x: (x==1).sum())
number_of_submissions = len(wecode_data_new)

# Tổng số problem đã solved (is_final = 1 && pre_score=10000) / Tổng số problem có submit
index_1 = problem_solved / number_of_submissions
print(index_1)

# merge vô train
d = index_1
train = train.merge(d,on="user_id")

user_id
1       0.000004
4       0.000140
5       0.000011
6       0.000321
7       0.000000
          ...   
1860    0.000106
1862    0.000015
1864    0.000106
1865    0.000038
1868    0.000102
Name: solved, Length: 1159, dtype: float64


In [ ]:
# Tổng số problem đã sumbit / Tổng số submissions 
problem_tries= groupByUserID['problem_id'].nunique()
number_of_submissions = len(wecode_data_new)
index_2=problem_tries/number_of_submissions

# merge vô train 
d = index_2
train = train.merge(d,on="user_id")


In [ ]:
train.columns = ["user_id","A1","A2","A3"]


- A1: Tổng pre_score của các submssion có is_final = 1
- A2: Tổng số problem đã solved (is_final = 1 && pre_score=10000) / Tổng số problem có submit
- A3: Tổng số problem đã sumbit / Tổng số submissions 
- A4:

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157 entries, 0 to 1156
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  1157 non-null   int64  
 1   A1       1157 non-null   int64  
 2   A2       1157 non-null   float64
 3   A3       1157 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 45.2 KB


In [ ]:
m = score[["id","CK"]]
m.columns = ["user_id","CK"]
train = train.merge(m,how = "left",on="user_id")
m.info()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  975 non-null    int64  
 1   CK       920 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 15.4 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157 entries, 0 to 1156
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  1157 non-null   int64  
 1   A1       1157 non-null   int64  
 2   A2       1157 non-null   float64
 3   A3       1157 non-null   float64
 4   CK       805 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 54.2 KB


In [ ]:
data_train = train.loc[ train.CK.notnull()]
data_train


,user_id,A1,A2,A3,CK
9,24,750299,0.000272,0.000295,9.5
10,27,39000,0.000011,0.000019,10.0
11,29,762760,0.000272,0.000302,8.5
13,34,822888,0.000306,0.000317,6.5
14,35,1477000,0.000548,0.000556,4.5
...,...,...,...,...,...
1149,1856,270790,0.000087,0.000117,7.5
1150,1857,386000,0.000140,0.000128,4.0
1151,1859,39000,0.000011,0.000102,1.5
1153,1862,40000,0.000015,0.000023,7.5


In [ ]:
x_train = data_train.drop(columns=['CK','user_id'], axis = 1)
y_train = data_train[["CK"]]

In [ ]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805 entries, 9 to 1155
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  805 non-null    int64  
 1   A1       805 non-null    int64  
 2   A2       805 non-null    float64
 3   A3       805 non-null    float64
 4   CK       805 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 37.7 KB


In [ ]:
x_train

,A1,A2,A3
9,750299,0.000272,0.000295
10,39000,0.000011,0.000019
11,762760,0.000272,0.000302
13,822888,0.000306,0.000317
14,1477000,0.000548,0.000556
...,...,...,...
1149,270790,0.000087,0.000117
1150,386000,0.000140,0.000128
1151,39000,0.000011,0.000102
1153,40000,0.000015,0.000023


In [ ]:
y_train

,CK
9,9.5
10,10.0
11,8.5
13,6.5
14,4.5
...,...
1149,7.5
1150,4.0
1151,1.5
1153,7.5


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x_train, y_train)


In [ ]:
data_test = train.loc[ train.CK.isnull()]
x_test = data_test.drop(columns=['CK','user_id'], axis = 1)



In [ ]:
y_pred = model.predict(x_test)

In [ ]:
table_pred =pd.DataFrame(data_test['user_id'], columns=['user_id'])
table_pred['CK'] = y_pred
table_pred.shape

(352, 2)

In [ ]:
table_pred.index = np.arange(0,len(table_pred))

In [ ]:
table_pred

,user_id,CK
0,1,5.120698
1,4,5.385555
2,5,5.119290
3,6,5.835433
4,7,5.199670
...,...,...
347,1852,5.140045
348,1855,5.956199
349,1860,5.670634
350,1864,6.566129
